## Presentación

Estos ejercicios tienen el fin de poder evaluar la situación en la que se encuentra cada unx de ustedes a esta altura de la materia. No es un examen, ni llevará ningún tipo de nota.

Les pedimos que respeten al pie de la letra el nombre de las funciones que les damos para completar y su "signature". Esto es, que respeten los argumentos de entrada y la salida. Esto es porque tenemos una forma de verificación automática que de otra manera no funciona. Cuando hayamos recibido los notebooks, haremos público el código de control.

Por supuesto, si quieren agregar celdas y más código, si quieren hacer pruebas para asegurarse de que el código funciona, etc., pueden hacerlo aquí mismo.

La _modalidad de entrega preferencial_ es a través de GitHub:
1. Hacer un fork del [repositorio](https://github.com/exord/UNSAM_IA), tocando el botón de arriba a la derecha (ver figura). 

2. Crear un subdirectorio dentro del directorio <tt>entregas</tt> con el formato "ApellidoNombre"
3. Completar este notebook y colocarlo en ese directorio. Hacer un "commit" y un "push" a su versión del repositorio.
4. Realizar un <tt>pull request</tt> desde GitHub y seguir las instrucciones. Más información en la [página de GitHub](https://help.github.com/en/github/collaborating-with-issues-and-pull-requests/about-pull-requests).

De ser necesario, en caso de sufrir de **alergia severa a GitHub**, el notebook puede enviarse por email.

La **fecha límite** para la entrega es el **martes 21 de abril a las 20 horas**.

<figure>
    <img src="https://github.com/exord/UNSAM_IA/blob/master/figures/fork_pullrequest.jpeg?raw=1", alt="Fork", width="800px", 
         style="float: left; margin-right: 10px;"/>
    
     <figcaption>Fig.1 - Ubicación de los botones para Fork y Pull request en Github.</figcaption>
</figure>

## 1. Probabilidad Condicional

In [0]:
#Si estas en Google Colab, ejecuta esta celda una unica vez:
!mkdir datasets
!mkdir datasets/student-alcohol-consumption
!wget https://raw.githubusercontent.com/exord/UNSAM_IA/master/datasets/student-alcohol-consumption/student-mat.csv
!mv student-mat.csv datasets/student-alcohol-consumption/student-mat.csv

Similar al ***Caso: Apruebo si falto a clase?*** del [notebook de probabilidad condicional](02_Condicional.ipynb), calcule la probabilidad de que un alumno haya faltado menos de 3 veces dado que sacó más de un 60% en la materia, i.e.
$$
P( \text{absences} < 3 | \text{G3} \geq 12).
$$

Por favor, utilice el siguiente signature. Se recomienda cambiar el código solamente entre los comentarios "Start" y "Finish". 
Si quiere probar cosas, siéntase libre de crear nuevas celdas de código, aunque estas no serán evaluadas.

In [0]:
import numpy as np
import pandas as pd

def prob_condicional(
  df=pd.read_csv('datasets/student-alcohol-consumption/student-mat.csv')):
    """Calcula P(absences < 3 | G3 >= 12).

    Input:
    df -- dataset de estudiantes

    Output:
    p        -- probabilidad de que absences < 3 dado que G3 >= 12
    """
    #Start
    #Ahora bien, yo se que P(absences < 3 | G3 >= 12) = P(absences < 3 y G3 >= 12) / P(G3 >= 12)
    #Calculo cada una de esas probabilidades
    amount_AyB = df[(df['absences'] < 3) & (df['G3'] >= 12)].shape[0]
    amount_B = df[(df['G3'] >= 12)].shape[0]
    total = df.shape[0]

    p_AyB = amount_AyB / total
    p_B = amount_B / total

    p_A_B = p_AyB / p_B

    #guarda el valor final, como un número, en esta variable
    resultado = p_A_B #si se quiere en porcentaje habria que multiplicar esto por 100, pero no estoy seguro que pide el problema
    
    #Finish
    return resultado

In [0]:
#### Test function ####
prob_condicional()

In [0]:
#### Asi lo empece a hacer, que es como la teorica, pero era mas complicado :p ####
import numpy as np
import pandas as pd

df=pd.read_csv('datasets/student-alcohol-consumption/student-mat.csv')

#Agrego columnas con un 1 o un 0 representando si cumple o no cada una de las condiciones pedidas
df['falto_poco'] = np.where(df['absences'] < 3, 1, 0)
df['buenas_notas'] = np.where(df['G3'] >= 12, 1, 0)

#Agrego columna auxiliar para contar filas
df['contar_aux'] = 1

#Nos quedamos con las columnas de interes
df = df[['falto_poco', 'buenas_notas', 'contar_aux']]

#Calculamos las cantidades
combinations = pd.pivot_table(
    df,
    values = 'contar_aux',
    index = ['falto_poco'],
    columns = ['buenas_notas'],
    aggfunc = np.sum,
    fill_value = 0)

combinations
#131, 81, 102, 81

## 2. Inferencia Bayesiana

Se tira una moneda N veces, de las cuales X sale cara. La moneda sale cara con una frecuencia $\mu$ desconocida, pero que se cree que es *fair*. 

Construya una función que recibe el vector con los N resultados (1: cara, 0: seca) y con él calcule tanto el posterior del parámetro $\mu$ como la probabilidad de que $\mu>0.7$. Use como prior una distribución normal con media 0.5 y desviación estandar 0.1 ($\mathcal{N}(0.5,0.1)$)

Por favor, utilice el siguiente signature. Se recomienda cambiar el código solamente entre los comentarios "Start" y "Finish". 
Si quiere probar cosas, siéntase libre de crear nuevas celdas de código, aunque estas no serán evaluadas.

<b>Ayuda</b>: usar los métodos <tt>pdf</tt> y <tt>pmf</tt> de las clases de <tt>scipy.stats</tt> que se importan al principio de la celda. Revisar también los notebooks de [teoría](03_Inferencia.ipynb) y [práctica](03p_Inferencia_Soluciones.ipynb) que se presentaron en clase.

In [0]:
import numpy as np
from scipy.stats import binom, norm

def inferencia_moneda(tiradas, seed=123):
    """Realiza inferencia bayesiana en el parametro mu de una moneda.

    Input:
    tiradas -- Resultado de N tiradas
    seed    -- Semilla para np.random (default=123)

    Output:
    p        -- probabilidad de que mu>0.7
    posterior-- posterior de mu, shape=(100,)
    """

    np.random.seed(seed)
    mu, step = np.linspace(0, 1, num = 100, retstep = True) #step es el espacio entre puntos

    #Numero de tiradas, y de veces que salio cara
    N = len(tiradas)
    X = np.sum(tiradas)

    #Start
    #Cambio el nombre a algunas variables (para facilidad mental nomas)
    mu_x = mu
    step_x = step
    cant_tiradas = N
    cant_caras = X

    #Calculo el prior
    prior = norm.pdf(mu_x, 0.5, 0.1)

    #Calculo la verosimilitud
    verosimilitud = binom.pmf(cant_caras, cant_tiradas, mu_x)

    #Calculo el pseudo posterior
    pseudo_posterior = prior * verosimilitud

    #Calculo la evidencia
    evidencia  = pseudo_posterior.sum() * step_x

    #Calculo el posterior
    posterior = pseudo_posterior / evidencia

    #Calculo la nueva probabilidad de que mu > 0.7
    np_mu_x = np.array(mu_x)
    np_posterior = np.array(posterior)
    probabilidad_mu_gt_07 = np_posterior[np_mu_x > 0.7].sum() * step_x

    p = probabilidad_mu_gt_07 #Probabilidad de que mu > 0.7

    #Finish
    assert len(posterior) == len(mu), "Tamaño del posterior distinto al de mu"
    return p, np.array(posterior)


In [0]:
#### Test function ####
#tiradas = np.random.randint(2, size=100)
tiradas = [1,1,1,1,1,0,1,1,1,1,0,1,1,1,0]
inferencia_moneda(tiradas)

In [0]:
#### Build function ####
import numpy as np
from scipy.stats import binom, norm

tiradas = np.random.randint(2, size=100)
seed = 123

np.random.seed(seed)
mu, step = np.linspace(0, 1, num=100, retstep=True) #step es el espacio entre puntos

#Numero de tiradas, y de veces que salio cara
N=len(tiradas)
X=np.sum(tiradas)

In [0]:
#Cambiar el nombre a algunas variables
mu_x = mu
cant_tiradas = N
cant_caras = X

#Calcular el prior
prior = norm.pdf(mu_x, 0.5, 0.1)

#Calculo la verosimilitud
verosimilitud = binom.pmf(cant_caras, cant_tiradas, mu_x)

#Calculo el pseudo posterior
pseudo_posterior = prior * verosimilitud

#Calculo la evidencia
evidencia  = pseudo_posterior.sum() * step

#Calculo el posterior
posterior = pseudo_posterior / evidencia

#Calculo la nueva probabilidad de que mu > 0.7
np_mu_x = np.array(mu_x)
np_posterior = np.array(posterior)
probabilidad_mu_gt_07 = np_posterior[np_mu_x > 0.7].sum()

In [0]:
from matplotlib import pylab as plt
plt.plot(mu_x, prior, label = "prior")
plt.plot(mu_x, verosimilitud, label = "verosimilitud")
plt.plot(mu_x, pseudo_posterior, label = "pseudo_posterior")
plt.plot(mu_x, posterior, label = "posterior")

plt.legend(loc = 0)

## 3. Regresión Lineal

Construye una función que dado un input (X_train, Y_train) realice un ajuste lineal de cuadrados mínimos, y con ello devuelva las predicciones para el input X_test como un vector Y_test.

Por favor, utilice el siguiente signature. Se recomienda cambiar el código solamente entre los comentarios "Start" y "Finish". 
Si quiere probar cosas, siéntase libre de crear nuevas celdas de código, aunque estas no serán evaluadas.

<b>Ayuda</b>: pueden usar la implementación de las ecuaciones normales que aparece en el notebook de [modelos lineales](05_ModelosLineales.ipynb) o las clases del módulo <tt>sklearn.linear_model</tt>.

In [0]:
import numpy as np

def ajuste_lineal(X_train, Y_train, X_test):
    """Calcula un ajuste lineal.

    Input:
    X_train -- datos de entrenamiento, shape=(N,1)
    Y_train -- etiquetas de entrenamiento, shape=(N,1)
    X_test  -- datos de testeo, shape=(N_test,1)

    Output:
    Y_test  -- prediccion de etiquetas para X_test, shape=(N_test,1)
    """

    N = len(X_train)
    N_test = len(X_test)

    #Start
    #Renombro algunas variables (para matchear codigo mio :p)
    x_train = X_train
    y_train = Y_train
    x_test = X_test

    #Ponemos nuestros datos en matrices
    x0_train = np.ones((len(x_train), 1))
    x_train = x_train.reshape(len(x_train), 1) #me aseguro que los vectores dados sean vector columna
    y_train = y_train.reshape(len(y_train), 1) #me aseguro que los vectores dados sean vector columna

    phi_train = np.hstack([x0_train, x_train])

    #Calculamos el predictor 
    phi_T_dot_phi = np.dot(phi_train.T, phi_train)
    phi_T_dot_y = np.dot(phi_train.T, y_train)

    w = np.linalg.solve(phi_T_dot_phi, phi_T_dot_y)

    #Calculamos la prediccion para el testing set
    x0_test = np.ones((len(x_test), 1))
    x_test = x_test.reshape(len(x_test), 1) #me aseguro que los vectores dados sean vector columna

    phi_test = np.hstack([x0_test, x_test])

    #Predigo los Y para el test
    y_test_predicted = np.dot(phi_test, w)
    
    #ahora guardamos los resultados en esta variable
    Y_test = y_test_predicted

    #Finish
    assert len(Y_test) == len(X_test), "Tamaño de Y_test distinto al de X_test"
    return np.array(Y_test)

In [0]:
#### Test function ####
import numpy as np
import numpy.random as rr
from matplotlib import pylab as plt

#Creamos datos aleatorios alrededor de una recta (simulo datos de algun experimento)
n = 100 #cantidad de X
x = rr.rand(n, 1)
x = x * 2 #esto amplia el rango (opcional, queda mas lindo el dibujo con el ruido por defecto)

#Definamos la recta y calculemos los Y (b y m son las incognitas que queremos predecir)
b = 4
m = 5
y = b + m * x

#Agregamos un poco de ruido a los Y (uso una distribucion normal para el ruido)
ruido = rr.randn(n, 1)
y = y + ruido

X_train = x
Y_train = y
X_test = x #no es ideal, pero es para un test rapido nomas

Y_test = ajuste_lineal(X_train, Y_train, X_test)

#Ploteo los datos originales y la recta donde ajustaron los datos predichos
plt.plot(x, y, '.')
plt.plot(X_test, Y_test, '-r')
plt.xlabel('X')
plt.ylabel('Y')

In [0]:
#### Build function ####
import numpy as np
import numpy.random as rr
## Creamos datos aleatorios alrededor de una recta (simulo datos de algun experimento)
#Calculamos los X
n = 100 #cantidad de X
x = rr.rand(n, 1)
x = x * 2 #esto amplia el rango (opcional, queda mas lindo el dibujo con el ruido por defecto)

#Definamos la recta y calculemos los Y (b y m son las incognitas que queremos predecir)
b = 4
m = 5
y = b + m * x

#Agregamos un poco de ruido a los Y (uso una distribucion normal para el ruido)
ruido = rr.randn(n, 1)
y = y + ruido

X_train = x
Y_train = y
X_test = x #no es ideal, pero es para un test rapido nomas

In [0]:
#Renombro algunas variables (para matchear codigo mio :p)
x_train = X_train
y_train = Y_train
x_test = X_test

#Ponemos nuestros datos en matrices
x0_train = np.ones((len(x_train), 1))
x_train = x_train.reshape(len(x_train), 1) #esto lo transforma en vector columna (era fila)
y_train = y_train.reshape(len(y_train), 1) #esto lo transforma en vector columna (era fila)

phi_train = np.hstack([x0_train, x_train])

#Calculamos el predictor 
phi_T_dot_phi = np.dot(phi_train.T, phi_train) 
phi_T_dot_y = np.dot(phi_train.T, y_train)

w = np.linalg.solve(phi_T_dot_phi, phi_T_dot_y)

#Calculamos la prediccion para el testing set
x0_test = np.ones((len(x_test), 1))
x_test = x_test.reshape(len(x_test), 1) #esto lo transforma en vector columna (era fila)

phi_test = np.hstack([x0_test, x_test])

#Predigo los Y para el test
y_test_predicted = np.dot(phi_test, w)